In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import  ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_length=20,
    max_token_limit=100,
    return_messages=True,

)

examples = [
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "movie": "Titanic",
        "answer": """
       🚢💑🧊
        """,
    },
    {
        "movie": "Inception",
        "answer": """
       🧠🌀🕰️
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]



example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert who explains the movie with three emojis."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
 

In [2]:
invoke_chain("What do you wnant to express about Inception?")

🧠🌀🕰️content='🧠🌀🕰️'


In [3]:
invoke_chain("What do you wnant to express about Jurassic Park")

🦖🌴🔬content='🦖🌴🔬'


In [4]:
invoke_chain("What is the movie I asked about first?")

The first movie you asked about was "Inception."content='The first movie you asked about was "Inception."'


In [5]:
invoke_chain("What do you wnant to express about Frozen?") 

❄️👑🎶The human asks what the AI wants to express about Inception. The AI responds with emojis representing the complexity and time-bending nature of the movie.content='❄️👑🎶'


In [6]:
invoke_chain("What do you wnant to express about Jaws?")

🦈🏖️🚤The human asks what the AI wants to express about Inception. The AI responds with emojis representing the complexity and time-bending nature of the movie. When asked about Jurassic Park, the AI responds with emojis of a dinosaur, palm trees, and a microscope.content='🦈🏖️🚤'


In [12]:
invoke_chain("What do you wnant to express about Star Wars?") 

🌌⚔️🤖The human asks what the AI wants to express about Inception, to which the AI responds with emojis representing the complexity and time-bending nature of the movie. When asked about Jurassic Park, the AI responds with emojis of a dinosaur, palm trees, and a microscope, confirming that the first movie discussed was "Inception." When asked about Frozen, the AI responds with ❄️👑🎶, and when asked about Jaws, the AI responds with 🦈🏖️🚤. The human then asks what the AI wants to express about Star Wars, and the AI responds with 🌌⚔️🤖. The human then asks what the movie they last discussed was, and the AI confirms it was "Inception." When asked about King Kong, the AI responds with 🦍🏢🌆.content='🌌⚔️🤖'


In [13]:
invoke_chain("What is the movie I asked about the first?") 

The first movie you asked about was "Inception."The human asks what the AI wants to express about Inception, to which the AI responds with emojis representing the complexity and time-bending nature of the movie. When asked about Jurassic Park, the AI responds with emojis of a dinosaur, palm trees, and a microscope, confirming that the first movie discussed was "Inception." When asked about Frozen, the AI responds with ❄️👑🎶, and when asked about Jaws, the AI responds with 🦈🏖️🚤. The human then asks what the AI wants to express about Star Wars, and the AI responds with 🌌⚔️🤖. The human then asks what the movie they last discussed was, and the AI confirms it was "Inception." When asked about King Kong, the AI responds with 🦍🏢🌆. The human then asks what the movie they last discussed was, and the AI confirms it was "Inception."content='The first movie you asked about was "Inception."'


In [14]:
invoke_chain("What do you wnant to express about King Kong?")

🦍🏢🌆The human asks what the AI wants to express about Inception, to which the AI responds with emojis representing the complexity and time-bending nature of the movie. When asked about Jurassic Park, the AI responds with emojis of a dinosaur, palm trees, and a microscope, confirming that the first movie discussed was "Inception." When asked about Frozen, the AI responds with ❄️👑🎶, and when asked about Jaws, the AI responds with 🦈🏖️🚤. The human then asks what the AI wants to express about Star Wars, and the AI responds with 🌌⚔️🤖. The human then asks what the movie they last discussed was, and the AI confirms it was "Inception." When asked about King Kong, the AI responds with 🦍🏢🌆. The human then asks what the movie they last discussed was, and the AI confirms it was "Inception." The human asks what the movie they discussed first was, and the AI confirms it was "Inception."content='🦍🏢🌆'


In [15]:
invoke_chain("What do you wnant to express about The Dark Knight?")

🦇🃏🌆The human asks what the AI wants to express about Inception, to which the AI responds with emojis representing the complexity and time-bending nature of the movie. When asked about Jurassic Park, the AI responds with emojis of a dinosaur, palm trees, and a microscope, confirming that the first movie discussed was "Inception." When asked about Frozen, the AI responds with ❄️👑🎶, and when asked about Jaws, the AI responds with 🦈🏖️🚤. The human then asks what the AI wants to express about Star Wars, and the AI responds with 🌌⚔️🤖. The human then asks what the movie they last discussed was, and the AI confirms it was "Inception." When asked about King Kong, the AI responds with 🦍🏢🌆. The human then asks what the movie they last discussed was, and the AI confirms it was "Inception." The human asks what the movie they discussed first was, and the AI confirms it was "Inception." The human asks what the AI wants to express about Star Wars, and the AI responds with 🌌⚔️🤖.content='🦇🃏🌆'
